In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn import metrics
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.read_csv("/content/drive/MyDrive/cassandra-udyam-2022/trainData.csv")
test_df=pd.read_csv("/content/drive/MyDrive/cassandra-udyam-2022/testData.csv")

In [4]:
dicty={"Jan":1, "Feb":2, "Mar":3, "Apr":4, "May":5, "Jun":6, "Jul":7, "Aug":8, "Sep":9, "Oct":10, "Nov":11, "Dec":12}

In [5]:
def cleaninig(df):
  
  rows=df.shape[0]

  #creating 3 arrays which stores day, month and year
  day=np.arange(rows)
  month=np.arange(rows)
  year=np.arange(rows)

  created=np.arange(rows)
  invoice=np.arange(rows)
  due=np.arange(rows)

  #splitting date into day, month and year for each 'Invoice_Date', 'Created_Date' and 'Due_Date'

  for row, h in df.Created.iteritems():
    time_stamp=str(h)
    A=time_stamp.split(' ')
    date=A[0]
    B=date.split('-')
    day[row]=int(B[0])
    month[row]=int(B[1])
    year[row]=int(B[2])
    year[row]-=2000

    #storing date into number of days
    created[row]=year[row]*365+month[row]*30+day[row]


  for row, h in df.Invoice_Date.iteritems():

    B=h.split('-')
    day[row]=int(B[0])
    month[row]=dicty[B[1]]
    year[row]=int(B[2])
    invoice[row]=year[row]*365+month[row]*30+day[row]

  for row, h in df.Due_Date.iteritems():

    B=h.split('-')
    day[row]=int(B[0])
    month[row]=dicty[B[1]]
    year[row]=int(B[2])
    due[row]=year[row]*365+month[row]*30+day[row]

    #Creating three bew columns representing the dates into number of days
 
  df['DUE']=due
  df['INVOICE']=invoice
  df['CREATED']=created

  #Creating two new column which are the differences of (due, invoice) and (invoive, created)

  due_diff=np.subtract(due, invoice)
  invoice_diff=np.subtract(invoice, created)
 

  df['due_diff']=due_diff
  df['invoice_diff']=invoice_diff

  #dropping 'Description' column as it contains 85% null values
  #dropping 'Created', 'Due_Date', 'Invoice_Date' as they are not in suitable format for model processing
  df.drop(['Created', 'Due_Date', 'Invoice_Date', 'Description'], axis=1, inplace=True)

  return df

In [6]:
df=cleaninig(df)
test_df=cleaninig(test_df)

In [7]:
df

,Vendor_Name,Amount,Settled,Outstanding,Number_of_Days_until_Payment,DUE,INVOICE,CREATED,due_diff,invoice_diff
0,David Taylor,672.78,672.78,0.0,13,4191,4161,4161,30,0
1,Stephen Wright MD,5101.98,5101.98,0.0,38,4218,4189,4189,29,0
2,Mark Cordova,7422.78,7422.78,0.0,61,4158,4129,4189,29,-60
3,Kimberly White,11.98,11.98,0.0,62,4189,4175,4234,14,-59
4,Teresa Marshall,5501.98,5501.98,0.0,2,4291,4290,4290,1,0
...,...,...,...,...,...,...,...,...,...,...
8748,Amy Sawyer,551.98,551.98,0.0,34,6534,6504,6536,30,-32
8749,Edward Santiago,386.98,386.98,0.0,34,6534,6504,6536,30,-32
8750,Angela Becker,279.81,279.81,0.0,5,6536,6536,6536,0,0
8751,Anna Carter,145.98,145.98,0.0,10,6541,6528,6537,13,-9


In [8]:
test_df

,Vendor_Name,Amount,Settled,Outstanding,DUE,INVOICE,CREATED,due_diff,invoice_diff
0,Aaron Hoffman,11.98,11.98,0.0,5865,5835,5831,30,4
1,Aaron Murphy,56.98,56.98,0.0,6146,6116,6167,30,-51
2,Aaron Parks,61.98,61.98,0.0,5944,5929,5929,15,0
3,Aaron Thomas,674.30,674.30,0.0,5021,5008,5012,13,-4
4,Abigail Hernandez,302.94,302.94,0.0,6394,6365,6371,29,-6
...,...,...,...,...,...,...,...,...,...
1120,Zachary Coleman,4201.98,4201.98,0.0,6122,6109,6111,13,-2
1121,Zachary Dudley,93.18,93.18,0.0,5354,5348,5348,6,0
1122,Zachary Hernandez,1255.98,1255.98,0.0,6481,6477,6480,4,-3
1123,Zachary Simmons,678.78,58.38,620.4,5536,5507,5507,29,0


In [9]:
#training data set
X = df.drop(['Number_of_Days_until_Payment', 'Vendor_Name'] , axis = 1)

#training target variable
Y = df['Number_of_Days_until_Payment']

print(X)
print(Y)

       Amount  Settled  Outstanding   DUE  INVOICE  CREATED  due_diff  \
0      672.78   672.78          0.0  4191     4161     4161        30   
1     5101.98  5101.98          0.0  4218     4189     4189        29   
2     7422.78  7422.78          0.0  4158     4129     4189        29   
3       11.98    11.98          0.0  4189     4175     4234        14   
4     5501.98  5501.98          0.0  4291     4290     4290         1   
...       ...      ...          ...   ...      ...      ...       ...   
8748   551.98   551.98          0.0  6534     6504     6536        30   
8749   386.98   386.98          0.0  6534     6504     6536        30   
8750   279.81   279.81          0.0  6536     6536     6536         0   
8751   145.98   145.98          0.0  6541     6528     6537        13   
8752   169.98   169.98          0.0  6570     6540     6541        30   

      invoice_diff  
0                0  
1                0  
2              -60  
3              -59  
4                0

In [10]:
#customized parameters for GradientBoostingRegressor

params= {"n_estimators": 800,
    "max_depth": 4,
    "min_samples_split": 6,
    "learning_rate": 0.07,
    "loss": "squared_error",
}

In [11]:
#using GradientBoostingRegressor 

model= GradientBoostingRegressor(**params)

In [12]:
model.fit(X, Y)

GradientBoostingRegressor(learning_rate=0.07, max_depth=4, min_samples_split=6,
                          n_estimators=800)

In [13]:
test_data = test_df.copy()

#test data set
test_data.drop('Vendor_Name', inplace=True, axis=1)

#predictions
predicts = model.predict(test_data)

In [14]:
model.score(X, Y)

0.5587789529302558

In [15]:
predicts = np.round(predicts)
print(predicts)

[-1. 69. 43. ... 15. 50. 42.]


In [16]:
#changing negative value in predictions to non-negative value as No_of_days_Until_Payment can't be negative
for i in range(predicts.size):
  if predicts[i]<0:
    predicts[i]=0
predicts

array([ 0., 69., 43., ..., 15., 50., 42.])

In [17]:
test_df

,Vendor_Name,Amount,Settled,Outstanding,DUE,INVOICE,CREATED,due_diff,invoice_diff
0,Aaron Hoffman,11.98,11.98,0.0,5865,5835,5831,30,4
1,Aaron Murphy,56.98,56.98,0.0,6146,6116,6167,30,-51
2,Aaron Parks,61.98,61.98,0.0,5944,5929,5929,15,0
3,Aaron Thomas,674.30,674.30,0.0,5021,5008,5012,13,-4
4,Abigail Hernandez,302.94,302.94,0.0,6394,6365,6371,29,-6
...,...,...,...,...,...,...,...,...,...
1120,Zachary Coleman,4201.98,4201.98,0.0,6122,6109,6111,13,-2
1121,Zachary Dudley,93.18,93.18,0.0,5354,5348,5348,6,0
1122,Zachary Hernandez,1255.98,1255.98,0.0,6481,6477,6480,4,-3
1123,Zachary Simmons,678.78,58.38,620.4,5536,5507,5507,29,0


In [18]:
dict={'Vendor_Name':test_df['Vendor_Name'], 'Number_of_Days_until_Payment':predicts}
result = pd.DataFrame(dict) 

In [19]:
result.to_csv('file.csv', index=False)

In [20]:
from google.colab import files
files.download("file.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>